In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
from scripts.benchmark import DATA_DIR, DATASETS, config_data,neg_log_likelihood

In [5]:
X_train, X_test, y_train, y_test = config_data("airfoil")
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


INFO:2026-01-15 14:06:43,262:jax._src.xla_bridge:810: Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.
2026-01-15 14:06:43,262 - jax._src.xla_bridge - INFO - Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.


(1202, 5)
(301, 5)
(1202,)
(301,)


In [6]:
reg = xgb.XGBRegressor(
    tree_method="hist",
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=0,
)

reg.fit(
    X_train, y_train,
    verbose=False,
)

pred = reg.predict(X_test)
rmse = root_mean_squared_error(y_test, pred)
print("rmse", root_mean_squared_error(y_test, pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, pred)).mean()}")

rmse 0.19470268487930298
nll -0.21734303442492023


In [17]:
from sklearn.ensemble import RandomForestRegressor
mlp= RandomForestRegressor(n_estimators=300)
mlp.fit(X_train,y_train)
mlp_pred= mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}")

rmse 0.2640629757201603
nll -0.14079819918795913


In [8]:
from sklearn.linear_model import LinearRegression
mlp = LinearRegression(fit_intercept=False)
mlp.fit(X_train,y_train)
mlp_pred = mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}")

rmse 0.6867814660072327
nll 0.5833855801014137


In [ ]:
from catboost import CatBoostRegressor
cat=CatBoostRegressor(depth=6, iterations=1000, learning_rate=0.05)
cat.fit(X_train,y_train)
cat_pred = cat.predict(X_test)
print("rmse", root_mean_squared_error(y_test, cat_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, cat_pred)).mean()}")

0:	learn: 0.9767437	total: 956us	remaining: 956ms
1:	learn: 0.9549973	total: 1.95ms	remaining: 973ms
2:	learn: 0.9310037	total: 2.71ms	remaining: 901ms
3:	learn: 0.9128200	total: 3.49ms	remaining: 869ms
4:	learn: 0.8899329	total: 4.24ms	remaining: 843ms
5:	learn: 0.8720766	total: 5ms	remaining: 828ms
6:	learn: 0.8562042	total: 5.64ms	remaining: 800ms
7:	learn: 0.8379556	total: 6.39ms	remaining: 792ms
8:	learn: 0.8199306	total: 7.01ms	remaining: 772ms
9:	learn: 0.8045970	total: 7.67ms	remaining: 759ms
10:	learn: 0.7870368	total: 8.35ms	remaining: 750ms
11:	learn: 0.7731777	total: 8.96ms	remaining: 738ms
12:	learn: 0.7562408	total: 9.66ms	remaining: 733ms
13:	learn: 0.7427784	total: 10.3ms	remaining: 726ms
14:	learn: 0.7301996	total: 10.9ms	remaining: 717ms
15:	learn: 0.7173943	total: 11.6ms	remaining: 711ms
16:	learn: 0.7036907	total: 12.4ms	remaining: 720ms
17:	learn: 0.6896145	total: 13.2ms	remaining: 721ms
18:	learn: 0.6761310	total: 13.9ms	remaining: 719ms
19:	learn: 0.6655755	total

In [10]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=[4, 16, 32, 63, 128, 256, 512, 256, 64, 8], solver="lbfgs", max_iter=5000)
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(
    f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}"
)

rmse 0.4299109280780521
nll 0.1773163174098252


c:\Users\vyron\.conda\envs\bde\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:602: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [18]:
from tabpfn import TabPFNRegressor
from tabpfn.constants import ModelVersion

tabpfn = TabPFNRegressor(random_state=2).create_default_for_version(
    ModelVersion.V2, ignore_pretraining_limits=True
)
tabpfn.fit(X_train, y_train)

tabpfn_pred = tabpfn.predict(X_test)

rmse = root_mean_squared_error(y_test, tabpfn_pred)
nll = neg_log_likelihood(y_test, tabpfn_pred, rmse).mean()

print("rmse", rmse)
print("nll", nll)

rmse 0.14475084841251373
nll -0.5138027585740881
